In [1588]:
%run update_data.py

In [1589]:
def six_mark_reward(match_count):
    if match_count == 6:
        print('中頭獎')
        return 8 * 1000 * 1000 # Jackpot – usually a very large and variable amount
    elif match_count == 5.5:  # 5 numbers + special number
        print('中二獎')
        return 40 * 1000  # Second Prize (variable, but using a typical value)
    elif match_count == 5:
        print('中三獎')
        return 20 * 1000  # Third Prize (variable, but using a typical value)
    elif match_count == 4.5:  # 4 numbers + special number
        return 9600  # Fourth Prize (fixed)
    elif match_count == 4:
        return 640  # Fifth Prize (fixed)
    elif match_count == 3.5:  # 3 numbers + special number
        return 320  # Sixth Prize (fixed)
    elif match_count == 3:
        return 40  # Seventh Prize (fixed)
    else:
        return 0  # handles invalid input

### Expected Value

In [1590]:
import math

def calculate_probability(n, k):
    """Calculates the combination C(n, k)."""
    return math.comb(n, k)  # More efficient than manual calculation

def calculate_expected_value():
    """Calculates the expected value of the Hong Kong Mark Six lottery."""
    total_combinations = calculate_probability(49, 6)
    expected_value = 0

    # Jackpot (6 numbers)
    prob_6 = 1 / total_combinations
    expected_value += prob_6 * six_mark_reward(6)

    # Second Prize (5 numbers + special number)
    prob_5_plus_special = calculate_probability(6, 5) * calculate_probability(1, 1) / total_combinations
    expected_value += prob_5_plus_special * six_mark_reward(5.5)

    # Third Prize (5 numbers)
    prob_5 = calculate_probability(6, 5) * calculate_probability(42, 1) / total_combinations #same as second prize
    expected_value += prob_5 * six_mark_reward(5)

    # Fourth Prize (4 numbers + special number)
    prob_4_plus_special = calculate_probability(6, 4) * calculate_probability(1, 1) / total_combinations
    expected_value += prob_4_plus_special * six_mark_reward(4.5)

    # Fifth Prize (4 numbers)
    prob_4 = calculate_probability(6, 4) * calculate_probability(42, 2) / total_combinations #same as fourth prize
    expected_value += prob_4 * six_mark_reward(4)

    # Sixth Prize (3 numbers + special number)
    prob_3_plus_special = calculate_probability(6, 3) * calculate_probability(1, 1) / total_combinations
    expected_value += prob_3_plus_special * six_mark_reward(3.5)

    # Seventh Prize (3 numbers)
    prob_3 = calculate_probability(6, 3) * calculate_probability(42, 3) / total_combinations #same as sixth prize
    expected_value += prob_3 * six_mark_reward(3)

    return expected_value

expected_value = calculate_expected_value()
print(f"Expected Value: {expected_value}")

中頭獎
中二獎
中三獎
Expected Value: 2.2082670424153177


# Simple LSTM Model

Click `Run All` button to see the result

## HYBER PARAMETER FOR LSTM MODEL HERE

In [1642]:
TARGET_YEAR = 2021 # Count the year since TARGET_YEAR (please note that in 2019 there is 6 month missing data due to COVID and its dropped in this notebook)
SEQ_LENGTH = 3 # Default is 7 for LSTM model (過去7期號碼)
TRAIN_SIZE = 0.8 # Default is 0.9 
LSTM_UNIT = 16 # Default is 50 // NN 內復雜性 
DROP_OUT = 0.5 # Default is 0
EPOCHS = 5
BATCH_SIZE = 8

L_TYPE = 'L1' # L1 or L2
L1_LAMBDA = 0.01 # 0.001 - 1
L2_LAMBDA = 0.01

## HYBER PARAMETER FOR LSTM MODEL HERE

In [1643]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.regularizers import l1, l2

In [1644]:
df = pd.read_json('data/data.json')

In [1645]:
df.tail(5)

,YEAR,TIMES,N1,N2,N3,N4,N5,N6,S1
5857,2025,11.0,14.0,18.0,30.0,36.0,40.0,42.0,1.0
5858,2025,12.0,3.0,13.0,15.0,18.0,19.0,24.0,37.0
5859,2025,13.0,1.0,4.0,18.0,25.0,38.0,45.0,23.0
5860,2025,14.0,1.0,7.0,21.0,27.0,33.0,45.0,17.0
5861,2025,15.0,5.0,15.0,20.0,26.0,30.0,40.0,39.0


In [1646]:
df.columns

Index(['YEAR', 'TIMES', 'N1', 'N2', 'N3', 'N4', 'N5', 'N6', 'S1'], dtype='object')

### FOR UPDATE THIS YEAR data (commented)

In [1647]:
# import requests
# from bs4 import BeautifulSoup
# import datetime

# current_year = datetime.datetime.now().year
# print(current_year)

# response = requests.get(f'http://www.nfd.com.tw/house/year/{current_year}.htm')
# response.encoding = 'big5'
# soup = BeautifulSoup(response.content, 'html.parser')
# table = soup.find('table')
# if table:
#     data = []
#     for row in table.find_all('tr'):
#         cells = row.find_all('td')
#         row_data = [cell.text.strip() for cell in cells]
#         data.append(row_data)     
# else:
#     print('error')   
# new_df = pd.DataFrame(data[1:], columns=data[0])
# filtered_df = df[df['YEAR'] != current_year]
# df = pd.concat([filtered_df, new_df], ignore_index=True)


### YEAR 2021, data has some issue

In [1648]:
df['YEAR'] = pd.to_numeric(df['YEAR'], errors='coerce') 
df = df[df['YEAR'] >= TARGET_YEAR]

# clean
df = df.dropna(subset=['YEAR'])

df = df[['N1', 'N2','N3', 'N4', 'N5', 'N6', 'S1']]
data = df.values

In [1649]:
# Scale the data (important for LSTMs)
scaler = MinMaxScaler(feature_range=(0, 1))
data = scaler.fit_transform(data)

In [1650]:
# Create sequences (e.g., use the last n draws [seq_length] to predict the next draw)
def create_sequences(data, seq_length):
    xs = []
    ys = []
    for i in range(len(data) - seq_length):
        x = data[i:(i + seq_length)]
        y = data[i + seq_length]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

In [1651]:
seq_length = SEQ_LENGTH  # How many past draws to use for prediction
X, y = create_sequences(data, seq_length)

In [1652]:
# Split data into training and testing sets
train_size = int(len(X) * TRAIN_SIZE)
X_train, X_test, y_train, y_test = X[:train_size], X[train_size:], y[:train_size], y[train_size:]

In [1653]:
scaler.inverse_transform(y_test)

array([[ 8., 20., 29., 33., 34., 44., 12.],
       [17., 21., 24., 27., 29., 48.,  8.],
       [11., 13., 17., 34., 40., 47., 21.],
       [11., 12., 18., 28., 29., 30., 39.],
       [17., 18., 31., 33., 39., 41., 14.],
       [12., 13., 24., 25., 27., 34.,  4.],
       [ 3., 16., 18., 26., 43., 45.,  9.],
       [ 6.,  7., 17., 20., 21., 25., 26.],
       [ 2.,  9., 16., 39., 43., 46., 20.],
       [ 5.,  6.,  8., 22., 26., 44., 40.],
       [ 3.,  4.,  7., 13., 27., 41., 38.],
       [ 3.,  6.,  7.,  8., 11., 29., 49.],
       [ 8., 12., 21., 35., 44., 45.,  2.],
       [ 7.,  9., 20., 21., 30., 35., 26.],
       [ 3., 16., 33., 35., 39., 44.,  8.],
       [ 6., 13., 19., 21., 26., 28.,  9.],
       [ 1.,  4., 16., 26., 39., 49.,  7.],
       [ 8.,  9., 16., 21., 32., 33., 34.],
       [ 9., 15., 19., 28., 31., 41., 42.],
       [ 2., 15., 23., 33., 34., 39., 19.],
       [ 6.,  7., 18., 24., 28., 29.,  1.],
       [ 2., 21., 34., 36., 41., 47., 31.],
       [ 2.,  3.,  7., 25., 46.,

### Please note that there is the special num

In [1654]:
special_num = [int(x[-1]) for x in scaler.inverse_transform(y_test)]

In [1655]:
print(special_num)

[12, 8, 21, 39, 13, 4, 9, 26, 20, 40, 38, 49, 2, 26, 8, 9, 6, 34, 42, 19, 1, 30, 13, 16, 27, 45, 10, 20, 45, 24, 13, 41, 19, 6, 21, 8, 35, 3, 23, 6, 27, 35, 2, 17, 6, 30, 23, 29, 12, 35, 34, 1, 38, 23, 22, 22, 4, 22, 38, 27, 10, 18, 45, 13, 22, 16, 11, 2, 6, 39, 27, 8, 45, 24, 1, 37, 23, 17, 39, 22, 16, 11, 2, 6, 39, 27, 8, 45, 24, 1, 37, 23, 17, 39, 22, 16, 11, 2, 6, 39, 27, 8, 45, 24, 1, 37, 23, 17, 39]


In [1656]:
# Build the LSTM model
model = Sequential()
if L_TYPE == 'L1':
    model.add(LSTM(LSTM_UNIT, input_shape=(X_train.shape[1], X_train.shape[2]), kernel_regularizer=l1(L1_LAMBDA))) 
else: 
    model.add(LSTM(LSTM_UNIT, input_shape=(X_train.shape[1], X_train.shape[2]), kernel_regularizer=l2(L2_LAMBDA))) 
model.add(Dropout(DROP_OUT))
model.add(Dense(y_train.shape[1])) # Output layer with the number of lottery numbers
model.compile(loss='mse', optimizer='adam') # Use mean squared error loss

# Train the model
model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, ) # Adjust epochs and batch size

Epoch 1/5


/Users/chutszkan/miniforge3/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.8112  
Epoch 2/5
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.5083  
Epoch 3/5
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.3442
Epoch 4/5
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 884us/step - loss: 0.2304
Epoch 5/5
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 852us/step - loss: 0.1470


In [1657]:
X_test.shape

(109, 3, 7)

In [1658]:
predictions = model.predict(X_test)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step


In [1659]:
real_predictions = np.round(scaler.inverse_transform(predictions)).astype(int)

# Simple way to handle the duplicate just add one or minor one 
def handle_predictions(all_predictions):
    
    new_all_predictions = []
    
    for predictions in all_predictions:
        """Ensures predictions are within 1-49 and unique, handling duplicates."""

        predictions = np.clip(predictions, 1, 49)  # Ensure range 1-49

        predictions = np.round(predictions).astype(int) # Convert to integers

        while len(set(predictions)) != len(predictions):  # While there are duplicates
            counts = np.bincount(predictions)  # Efficiently count occurrences
            duplicates = np.where(counts > 1)[0]  # Get the duplicate values

            for duplicate_num in duplicates:
                duplicate_indices = np.where(predictions == duplicate_num)[0]
                for i in duplicate_indices[1:]:  # Replace all but the first
                    while True:
                        new_num = np.random.randint(1, 50)  # 1 to 49 inclusive
                        if new_num not in predictions:
                            predictions[i] = new_num
                            break  # Found a unique replacement
        new_all_predictions.append(predictions)

    return new_all_predictions
        
new_real_predictions = handle_predictions(real_predictions)

In [1660]:
# print(new_real_predictions)

In [1661]:
y_test = np.round(scaler.inverse_transform(y_test)).astype(int)

In [1662]:
# print(y_test)

In [1663]:
result = []
for nth in range(len(y_test)):
    num_correct = 0
    pred_list = real_predictions[nth]
    ground_truth_list = y_test[nth]
    special_number = special_num[nth]
    for x in pred_list:
        if x in ground_truth_list:
            print
            if int(x) in [int(special_number)]:
                print('中特別號碼')
                num_correct += 0.5
            else:
                num_correct += 1
    result.append(num_correct)

中特別號碼
中特別號碼
中特別號碼
中特別號碼
中特別號碼
中特別號碼


## For the result, 買1組 10蚊 (7個數字 -> 70)

In [1664]:
result = [float(r) for r in result]

In [1665]:
print(f'buying: {len(result)} tickets')

total_spend = len(result) * 70
print(f'Spending: ${len(result) * 70}')

total_reward = sum([six_mark_reward(x) for x in result])
print(f'Total Reward: ${total_reward}')

final_earn = total_reward - total_spend
print(f'Final earn: {final_earn}')

print(f'Earn Ratio: {final_earn/(len(result) * 10) * 100:.2f}%')

buying: 109 tickets
Spending: $7630
Total Reward: $680
Final earn: -6950
Earn Ratio: -637.61%


### Predict the future 🤑

In [1666]:
seq_length = SEQ_LENGTH
last_sequence = data[-seq_length:] 
last_sequence = np.reshape(last_sequence, (1, seq_length, X_train.shape[2]))
last_sequence_scaled = scaler.transform(last_sequence.reshape(seq_length, X_train.shape[2])).reshape(1, seq_length, X_train.shape[2])
next_prediction = model.predict(last_sequence_scaled)
next_prediction = scaler.inverse_transform(next_prediction)
next_prediction = np.round(next_prediction).astype(int)
print("Predicted next draw:", handle_predictions(next_prediction))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Predicted next draw: [array([ 5,  7,  9, 15, 18, 26, 33])]


### Ground Truth

In [1667]:
print(y_test)

[[ 8 20 29 33 34 44 12]
 [17 21 24 27 29 48  8]
 [11 13 17 34 40 47 21]
 [11 12 18 28 29 30 39]
 [17 18 31 33 39 41 14]
 [12 13 24 25 27 34  4]
 [ 3 16 18 26 43 45  9]
 [ 6  7 17 20 21 25 26]
 [ 2  9 16 39 43 46 20]
 [ 5  6  8 22 26 44 40]
 [ 3  4  7 13 27 41 38]
 [ 3  6  7  8 11 29 49]
 [ 8 12 21 35 44 45  2]
 [ 7  9 20 21 30 35 26]
 [ 3 16 33 35 39 44  8]
 [ 6 13 19 21 26 28  9]
 [ 1  4 16 26 39 49  7]
 [ 8  9 16 21 32 33 34]
 [ 9 15 19 28 31 41 42]
 [ 2 15 23 33 34 39 19]
 [ 6  7 18 24 28 29  1]
 [ 2 21 34 36 41 47 31]
 [ 2  3  7 25 46 48 13]
 [ 5 24 26 32 35 40 16]
 [ 1 12 16 20 21 30 27]
 [14 18 20 25 32 43 45]
 [ 5 18 21 24 27 44 10]
 [ 5 18 21 24 44 47 20]
 [ 2  3 14 25 27 36 45]
 [ 9 17 28 31 40 49 25]
 [ 2  4 29 35 38 49 14]
 [ 8 15 21 32 43 47 41]
 [10 16 24 32 37 40 19]
 [10 33 35 40 44 48  6]
 [ 4  8  9 19 20 48 21]
 [19 36 37 43 44 49  8]
 [11 17 22 26 33 46 35]
 [13 21 28 30 41 49  3]
 [ 5  6 13 15 36 49 23]
 [ 7 13 16 22 23 31  6]
 [14 16 25 40 41 45 27]
 [ 7 11 13 24 27

### Predict

In [1668]:
print(new_real_predictions)

[array([ 7, 13, 19, 25, 31, 38, 20]), array([ 7, 13, 20, 25, 33, 40, 21]), array([ 7, 13, 19, 25, 32, 39, 20]), array([ 6, 13, 19, 25, 32, 39, 20]), array([ 7, 13, 19, 25, 32, 39, 20]), array([ 7, 13, 19, 25, 32, 39, 20]), array([ 7, 13, 18, 23, 31, 38, 19]), array([ 6, 12, 18, 23, 30, 38, 19]), array([ 6, 12, 17, 23, 29, 36, 18]), array([ 6, 13, 19, 24, 31, 38, 44]), array([ 6, 13, 19, 25, 31, 38, 23]), array([ 7, 13, 19, 26, 31, 39, 20]), array([ 6, 12, 17, 26, 30, 37, 18]), array([ 6, 12, 18, 24, 30, 37, 19]), array([ 6, 12, 18, 24, 30, 37, 19]), array([ 7, 13, 19, 24, 31, 39, 20]), array([ 7, 12, 18, 23, 30, 37, 19]), array([ 7, 12, 18, 23, 30, 37, 19]), array([ 7, 12, 17, 23, 29, 37, 18]), array([ 7, 13, 19, 25, 31, 39, 20]), array([ 7, 13, 19, 25, 32, 39, 20]), array([ 7, 13, 19, 24, 31, 38,  5]), array([ 7, 13, 19, 24, 31, 39, 20]), array([ 7, 13, 19, 24, 31, 38, 20]), array([ 7, 13, 20, 25, 32, 40, 26]), array([ 6, 12, 18, 24, 30, 38, 19]), array([ 6, 13, 18, 25, 31, 38, 19]), 

### Next Prediction

In [1669]:
print("Predicted next draw:", handle_predictions(next_prediction))

Predicted next draw: [array([ 5,  7,  9, 15, 18, 26, 48])]


### For loop until the model can get at least 三獎

In [1670]:
import itertools

In [1671]:
from tensorflow.keras.callbacks import EarlyStopping

In [1672]:
TARGET_YEAR = 2024
TRAIN_SIZE = 0.999 # for training the last one draw

# Hyperparameter ranges
SEQ_LENGTHS = range(2, 11)  # 2 to 10 inclusive
LSTM_UNITS = [8, 16, 32, 64]
DROP_OUTS = [i / 10 for i in range(1, 9)]  # 0.1 to 0.8
EPOCHSS = [2, 5, 10, 20, 50] #2-50
BATCH_SIZES = [4, 8, 16, 32, 64]
L_TYPES = ['L1', 'L2']
L1_LAMBDAS = [0.001, 0.01, 0.1]
L2_LAMBDAS = [0.001, 0.01, 0.1]

In [1673]:
best_reward = -np.inf  # Initialize with negative infinity
best_model = None
best_hyperparameters = None

In [1674]:
import json
import datetime
import tensorflow as tf

In [1675]:
string_now = datetime.datetime.now().strftime("%Y-%m-%d-%H:%M:%S")
string_now

'2025-02-13-03:05:51'

In [1676]:
def append_to_json(filepath, new_data):
    try:
        os.makedirs(os.path.dirname(filepath), exist_ok=True)  # Create dir if needed

        try:  # Try to open and load. If not exists or empty/invalid, start fresh
            with open(filepath, 'r') as f:  # Open in read mode first
                file_data = json.load(f)
        except (FileNotFoundError, json.JSONDecodeError):  # File DNE or invalid JSON
            file_data = []  # Initialize an empty list

        # Append the new data
        if isinstance(new_data, list):
            file_data.extend(new_data)
        else:
            file_data.append(new_data)

        with open(filepath, 'w') as f:  # Open in write mode to overwrite
            json.dump(file_data, f, indent=4)  # Correct way to write
        print(f"Data appended to/saved in {filepath}")

    except Exception as e:
        print(f"An error occurred: {e}")

In [1677]:
import os

In [ ]:
for SEQ_LENGTH, LSTM_UNIT, DROP_OUT, EPOCHS, BATCH_SIZE, L_TYPE, L1_LAMBDA, L2_LAMBDA in itertools.product(
    SEQ_LENGTHS, LSTM_UNITS, DROP_OUTS, EPOCHSS, BATCH_SIZES, L_TYPES, L1_LAMBDAS, L2_LAMBDAS
):
    record_results = []
    print(f"Testing: SEQ_LENGTH={SEQ_LENGTH}, LSTM_UNIT={LSTM_UNIT}, DROP_OUT={DROP_OUT}, EPOCHS={EPOCHS}, BATCH_SIZE={BATCH_SIZE}, L_TYPE={L_TYPE}, L1_LAMBDA={L1_LAMBDA}, L2_LAMBDA={L2_LAMBDA}")

    data = df.values  # Assuming df is your DataFrame
    scaler = MinMaxScaler(feature_range=(0, 1))
    data = scaler.fit_transform(data)

    X, y = create_sequences(data, SEQ_LENGTH)
    train_size = int(len(X) * TRAIN_SIZE)
    X_train, X_test, y_train, y_test = X[:train_size], X[train_size:], y[:train_size], y[train_size:]

    special_num = [int(x[-1]) for x in scaler.inverse_transform(y_test)]

    model = Sequential()
    if L_TYPE == 'L1':
        model.add(LSTM(LSTM_UNIT, input_shape=(X_train.shape[1], X_train.shape[2]), kernel_regularizer=l1(L1_LAMBDA)))
    else:
        model.add(LSTM(LSTM_UNIT, input_shape=(X_train.shape[1], X_train.shape[2]), kernel_regularizer=l2(L2_LAMBDA)))
    model.add(Dropout(DROP_OUT))
    model.add(Dense(y_train.shape[1]))
    model.compile(loss='mse', optimizer='adam')

    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    try:  # Handle potential training errors
        history = model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_split = 0.2, callbacks=[early_stopping], verbose = 0)  # Use EarlyStopping
        predictions = model.predict(X_test, verbose = 0)
        real_predictions = np.round(scaler.inverse_transform(predictions)).astype(int)
        new_real_predictions = handle_predictions(real_predictions)

        y_test = np.round(scaler.inverse_transform(y_test)).astype(int)

        result = []
        for nth in range(len(y_test)):
            num_correct = 0
            pred_list = new_real_predictions[nth] # Use corrected predictions
            ground_truth_list = y_test[nth]
            special_number = special_num[nth]
            for x in pred_list:
                if x in ground_truth_list:
                    if int(x) in [int(special_number)]:
                        num_correct += 0.5
                    else:
                        num_correct += 1
            result.append(num_correct)

        total_reward = sum([six_mark_reward(x) for x in result])
        
        last_sequence = data[-seq_length:] 
        last_sequence = np.reshape(last_sequence, (1, seq_length, X_train.shape[2]))
        last_sequence_scaled = scaler.transform(last_sequence.reshape(seq_length, X_train.shape[2])).reshape(1, seq_length, X_train.shape[2])
        next_prediction = model.predict(last_sequence_scaled)
        next_prediction = scaler.inverse_transform(next_prediction)
        next_prediction = np.round(next_prediction).astype(int)
        
        record_results.append({
            'SEQ_LENGTH': SEQ_LENGTH,
            'LSTM_UNIT': LSTM_UNIT,
            'DROP_OUT': DROP_OUT,
            'EPOCHS': EPOCHS,
            'BATCH_SIZE': BATCH_SIZE,
            'L_TYPE': L_TYPE,
            'L1_LAMBDA': L1_LAMBDA,
            'L2_LAMBDA': L2_LAMBDA,
            'total_reward': total_reward,
            'next_prediction': str(handle_predictions(next_prediction)),
            'best_epoch': early_stopping.best_epoch, # Record best epoch
            'val_loss': min(history.history['val_loss']) if 'val_loss' in history.history else None # Record val loss
        })
        
        current_reward = total_reward
        if current_reward > best_reward:
            best_reward = current_reward
            best_model = model  # Save the model itself
            best_model.save("record/lstm/best_lottery_model.h5")  # Save in HDF5 format
            
    except Exception as e:
        print(f"Error during training: {e}")
        record_results.append({
            'SEQ_LENGTH': SEQ_LENGTH,
            'LSTM_UNIT': LSTM_UNIT,
            'DROP_OUT': DROP_OUT,
            'EPOCHS': EPOCHS,
            'BATCH_SIZE': BATCH_SIZE,
            'L_TYPE': L_TYPE,
            'L1_LAMBDA': L1_LAMBDA,
            'L2_LAMBDA': L2_LAMBDA,
            'total_reward': None,
            'next_prediction': None,
            'best_epoch': None,
            'val_loss': None,
            'error': str(e)  # Record the error
        })
    append_to_json(filepath=f'record/lstm/{string_now}.json', new_data=record_results)

Testing: SEQ_LENGTH=2, LSTM_UNIT=8, DROP_OUT=0.1, EPOCHS=2, BATCH_SIZE=4, L_TYPE=L1, L1_LAMBDA=0.001, L2_LAMBDA=0.001


/Users/chutszkan/miniforge3/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step


Data appended to/saved in record/lstm/2025-02-13-03:05:51.json
Testing: SEQ_LENGTH=2, LSTM_UNIT=8, DROP_OUT=0.1, EPOCHS=2, BATCH_SIZE=4, L_TYPE=L1, L1_LAMBDA=0.001, L2_LAMBDA=0.01


/Users/chutszkan/miniforge3/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

/Users/chutszkan/miniforge3/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
Data appended to/saved in record/lstm/2025-02-13-03:05:51.json
Testing: SEQ_LENGTH=2, LSTM_UNIT=8, DROP_OUT=0.4, EPOCHS=2, BATCH_SIZE=64, L_TYPE=L2, L1_LAMBDA=0.001, L2_LAMBDA=0.001


/Users/chutszkan/miniforge3/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
Data appended to/saved in record/lstm/2025-02-13-03:05:51.json
Testing: SEQ_LENGTH=2, LSTM_UNIT=8, DROP_OUT=0.4, EPOCHS=2, BATCH_SIZE=64, L_TYPE=L2, L1_LAMBDA=0.001, L2_LAMBDA=0.01


/Users/chutszkan/miniforge3/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
